In [91]:
import pandas as pd
import numpy as np
from numpy import random as r
from matplotlib import pyplot as plt

In [92]:
path="/home/stan/Documents/dev/psignal/Data/"
pureTwt='twitter_noretweets_daily.csv'
ptwt=pd.read_csv(filepath_or_buffer=path+pureTwt,delimiter=",",header=0)
ptwt=ptwt.sample(n=1000)
ptwt.to_csv("sample ptwt.csv")

In [95]:
sampleTWT='sample ptwt.csv'
ptwt=pd.read_csv(filepath_or_buffer='/home/stan/Documents/dev/psignal/'+sampleTWT,delimiter=",",header=0)

In [96]:
ptwt['TIMESTAMP_UTC']=pd.to_datetime(arg=ptwt.TIMESTAMP_UTC,infer_datetime_format='true')
cols=['SYMBOL','TIMESTAMP_UTC']
ptwt=ptwt.reindex(columns=cols)
ptwt.columns=['sym','date']
ptwt['return']=r.randn(len(mini_twt))

In [97]:
ptwt.head()

,sym,date,return
0,NVS,2014-05-07 04:00:00,-0.022227
1,AET,2014-06-04 04:00:00,-0.143822
2,FLIC,2013-04-06 04:00:00,-0.932955
3,ALE,2015-01-16 05:00:00,1.546380
4,GSAT,2013-01-02 05:00:00,-0.561737


In [126]:
sp500=pd.DataFrame({'date':pd.date_range(start='2010-01-01',end='2015-12-31',frequency="D"),\
                    'return':r.randn(len(pd.date_range(start='2010-01-01',end='2015-12-31',frequency="D")))})
sp500.head()

,date,return
0,2010-01-01,-0.078026
1,2010-01-02,0.921009
2,2010-01-03,-0.569918
3,2010-01-04,0.352940
4,2010-01-05,1.514005


In [120]:
windows=pd.period_range(start='2010',end='2015',freq="M")
windows=windows.format()
windows[0]

'2010-01'

In [ ]:
import <regression> as reg
for stock in ptwt['sym'].unique():
    for window_end_idx in range(1,len(windows)):
        mask=(ptwt['sym']==stock)&(ptwt['date']>=windows[window_end_idx-1])&(ptwt['date']<windows[window_end_idx])
        tmp=ptwp[mask]
        r_t_plus_one=tmp['return'].ix[1:]
        r_t=tmp['return'].ix[0:len(tmp)-1]
        mask_market_index=(ptwt['date']>=windows[window_end_idx-1])&(ptwt['date']<windows[window_end_idx])
        market_index=SP500[mask].ix[0:len(tmp)-1] #check and correct for stationarity
        SIR=ptwt[mask_market_index].groupby('cluster','date').mean() #compute mean return for each day and cluster
        stationarity_test=reg.adftest(r_t_plus_one, kind="c")
        while (stationarity_test):
            r_t_plus_one=reg.takeFirstDifference(r_t_plus_one)
            r_t=reg.takeFirstDifference(r_t)
            stationarity_test=reg.adftest(r_t_plus_one, kind="c")
        tmp_reg=reg.GLS(y=r_t_plus_one,x=[r_t,market_index],include_constant=True,return_residuals=True)
        tmp_reg_prime=reg.GLS(y=tmp_reg.residuals,x=[r_t,SIR],include_constant=True,return_residuals=True)
        regression[(stock, window)]=(tmp_reg,tmp_reg_prime)

In [122]:
mask=(ptwt['sym']=='ALE')&(ptwt['date']>='2015-1')&(ptwt['date']<='2015-12')
ptwt[mask]

,sym,date,return
3,ALE,2015-01-16 05:00:00,1.54638
